### Implementing an Autoencoder in Pytorch
Autoencoder는 입력을 출력으로 복사하는 신경망
hidden layer의 뉴런 수를 input layer 보다 작게해 데이터를 압축(차원 축소)한다던지, input data에 noise를 추가해 원본 입력을 복원하도록 학습시키는 등의 다양한 오토 인코더 존재

In [1]:
import torch
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

### MNIST 데이터 셋을 활용해 직접 Auto Encoder를 구현해 보겠습니다.

In [2]:
tensor_transform = transforms.ToTensor()
dataset = datasets.MNIST(root = "./data",
                       train = True,
                       download = True,
                       transform = tensor_transform)
loader = torch.utils.data.DataLoader(dataset = dataset, batch_size=1, shuffle=True)

In [3]:
print(loader)

### create autoencoder class

In [4]:
class AE(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(28 * 28, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 18)
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(18, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 28*28),
            torch.nn.Sigmoid()
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [5]:
model = AE()
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                            lr = 1e-1,
                            weight_decay = 1e-8)

In [6]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device=torch.device("mps:0" if torch.backends.mps.is_available() else "cuda:0")

epochs = 20
outputs = []
losses = []
for epoch in range(epochs):
    for (image, _) in loader:
        image = image.reshape(-1, 28*28)
        # reshape
        reconstructed = model(image)
        # output of AE
        
        loss = loss_function(reconstructed, image)
        # calculate loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.detach().numpy())
        # store the loss in a list for plotting
    outputs.append((epochs, image, reconstructed))
    print(f"({epoch + 1} / {epochs}) completed")
    
plt.style.use('fivethirtyeight')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.plot(losses[-100:])

KeyboardInterrupt: 

In [ ]:
print(len(outputs[19][1][5]))

In [ ]:
# plt.imshow(model(dataset[5]))


for i, item in enumerate(image):
    item = item.reshape(-1, 28, 28).detach().numpy()
    if i == 0:
        plt.imshow(item[0])

In [ ]:
plt.imshow(outputs[19][2][9].reshape(28, 28).detach().numpy())

# for i, item in enumerate(reconstructed):
#     item = item.reshape(-1, 28, 28).detach().numpy()
#     if i == 9:
#         plt.imshow(item[0])

In [ ]:
imaged = reconstructed[19].detach().numpy() 
image_reshaped = imaged.reshape(28, 28)
# image_reshaped.shape
plt.figure(figsize = (4, 4))
plt.imshow(image_reshaped, cmap="gray")
plt.show()